# 🚀 Multi-Agent Sequential Orchestration (Microsoft Agent Framework)

This notebook shows how to build a **sequential multi‑agent workflow** with the **Microsoft Agent Framework**. It includes:
- Clear sectioning and professional comments
- A bullet architecture diagram with Unicode arrows (↓)
- Notebook‑safe async execution (`await main()`)


## 📐 Architecture Overview 

- Customer Feedback
  ↓
- Summarizer Agent
  ↓
- Classifier Agent
  ↓
- Action Recommendation Agent
  ↓
- Final Output


## ✅ Prerequisites

1. Azure project with a chat model deployment you can call (e.g., Azure OpenAI).
2. Environment variables available to this notebook session:
   - `PROJECT_ENDPOINT` → your Azure AI project endpoint
   - `MODEL_DEPLOYMENT_NAME` → your chat model deployment name (e.g., `gpt-4o-mini`)
3. Authentication: use `DefaultAzureCredential` (works with `az login` locally or Managed Identity in Azure).


# 📦 1) Import Required Libraries

In [1]:
# Standard libs
import os
from dotenv import load_dotenv
import asyncio
from typing import cast

# Azure identity for authentication
from azure.identity import DefaultAzureCredential

# Microsoft Agent Framework core types
from agent_framework import ChatMessage, Role, SequentialBuilder, WorkflowOutputEvent, ChatAgent

# Azure AI client wrapper for Agent Framework
from agent_framework.azure import AzureAIAgentClient


# 🔧 2) Load Environment Variables & Validate Configuration

In [2]:
# Load .env if present (handy in local dev)
load_dotenv()

# Fail-fast accessor to keep configuration obvious
def get_env(name: str) -> str:
    value = os.getenv(name)
    if not value:
        raise RuntimeError('Missing required env var: ' + name)
    return value

endpoint = get_env('PROJECT_ENDPOINT')
model = get_env('MODEL_DEPLOYMENT_NAME')

print('Environment OK')


Environment OK


# 🤖 3) Create Azure AI Agent Client

In [4]:
# Uses DefaultAzureCredential (supports az login, managed identity, etc.)
credential = DefaultAzureCredential()
chat_client = AzureAIAgentClient(
    credential=credential,
    model_deployment_name=model,
    project_endpoint=endpoint
)
print('Chat Client ready')


Chat Client ready


# 🧠 4) Define Specialized Agents (Prompts kept concise & deterministic)

In [7]:
# Keep each instruction short and outcome-focused for more deterministic behavior
summarizer_instructions = (
    'Summarize the customer feedback in one short sentence. '
    'Keep it neutral and concise. '
    'Example: App crashes during upload.'
)

classifier_instructions = (
    'Classify the feedback as exactly one of: '
    'Positive, Negative, Feature request. '
    'Return ONLY the category.'
)

action_instructions = (
    'Based on the summary and classification, suggest a next action. '
    'One sentence, must contain a verb, max 12 words.'
)

# Create agents
a_summarizer = ChatAgent(chat_client=chat_client, name='summarizer', instructions=summarizer_instructions)
a_classifier = ChatAgent(chat_client=chat_client, name='classifier', instructions=classifier_instructions)
a_action = ChatAgent(chat_client=chat_client, name='action', instructions=action_instructions)

agents = [a_summarizer, a_classifier, a_action]
print('Agents defined: ' + ', '.join([a.name for a in agents]))

Agents defined: summarizer, classifier, action


# 📝 5) Provide Input Feedback (sample text)

In [8]:
# Use any feedback string you'd like to test. Kept on a single line to avoid escaping issues.
feedback = (
    'I reached out to your customer support yesterday because I could not access my account. '
    'The representative responded quickly, was polite, and resolved the issue. '
    'One of the best support experiences I have had.'
)
print('Sample feedback ready')

Sample feedback ready


# 🔗 6) Build the Sequential Workflow

In [9]:
# Order matters: output from each agent flows to the next
workflow = SequentialBuilder().participants(agents).build()
print('Sequential workflow built')

Sequential workflow built


# ▶️ 7) Execute the Workflow (Notebook-Safe Async)

In [ ]:
# Important: In notebooks, DO NOT use asyncio.run(). Use `await main()` instead.

async def main():
    outputs = []
    async for event in workflow.run_stream('Customer feedback: ' + feedback):
        if isinstance(event, WorkflowOutputEvent):
            outputs.append(cast(list[ChatMessage], event.data))

    if outputs:
        print('============= FINAL OUTPUT =============')
        for i, msg in enumerate(outputs[-1], start=1):
            name = msg.author_name or ('assistant' if msg.role == Role.ASSISTANT else 'user')
            print('-' * 60)
            # Use format() (not f-strings) to avoid any parser confusion in some tools
            print('\n{:02d} [{}]\n{}'.format(i, name, msg.text))

# In Jupyter/VS Code notebooks, this will run under the existing event loop
await main()

============= FINAL OUTPUT =============
------------------------------------------------------------

01 [user]
Customer feedback: I reached out to your customer support yesterday because I could not access my account. The representative responded quickly, was polite, and resolved the issue. One of the best support experiences I have had.
------------------------------------------------------------

02 [summarizer]
Customer praised the support for prompt, polite service and resolving account access issue.
------------------------------------------------------------

03 [classifier]
Positive
------------------------------------------------------------

04 [action]
Encourage the support team to maintain their high service standards.


# 🧪 8) Troubleshooting & Tips

- **Auth:** If authentication fails, run `az login` locally or configure Managed Identity when hosted.
- **Env Vars:** If you see `Missing required env var`, set `PROJECT_ENDPOINT` and `MODEL_DEPLOYMENT_NAME` in your environment or a `.env` file.
- **Prompt hygiene:** Keep prompts short, define allowed outputs, and include examples.
- **Observability:** For debugging, log intermediate `WorkflowOutputEvent` steps (already collected here in `outputs`).
